In [104]:
import numpy as np
from collections import deque
mat = np.array([[0, 1, 1, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 0]])

expected_dist = np.array([[0,1,1,2,3,3,3],
                          [1,0,1,1,2,2,2],
                          [1,1,0,2,3,3,3],
                          [2,1,2,0,1,1,1],
                          [3,2,3,1,0,2,1],
                          [3,2,3,1,2,0,1],
                          [3,2,3,1,1,1,0]])

In [116]:
num_vertices = mat.shape[0]
res = np.full((num_vertices, num_vertices), np.inf)

# Finish this loop
for i in range(num_vertices):
    Q = deque()
    # initialize visited vector
    visited = np.full((num_vertices), False)
    
    # set diagonal elements to distance of 0
    np.fill_diagonal(res,0)
    
    # set first vertex in queue and set as visited
    Q.append([i, 0])
    visited[i] = True

    # run through network while there are still vertices left in the queue
    while Q:
        # current reference distance
        distance = Q[0][1]

        # determine adjacent vertices
        adj_vert = np.where(mat[Q[0][0]] > 0)[0]
        
        # run if there exists any adjacent vertices
        if adj_vert.any():
            
            # filter out vertices that have already been visited
            adj_vert = [vert for vert in adj_vert if visited[vert] == False]
            
            # add adjacent vertices to queue and set them as visited
            [Q.append([vert,distance + 1]) for vert in adj_vert]
            np.put(visited, adj_vert, True)

            # record the distance to the vertices
            np.put(res[i], adj_vert,distance+1)
            
            # remove current vertex from queue
            Q.popleft()

print(res)

[[0. 1. 1. 2. 3. 3. 3.]
 [1. 0. 1. 1. 2. 2. 2.]
 [1. 1. 0. 2. 3. 3. 3.]
 [2. 1. 2. 0. 1. 1. 1.]
 [3. 2. 3. 1. 0. 2. 1.]
 [3. 2. 3. 1. 2. 0. 1.]
 [3. 2. 3. 1. 1. 1. 0.]]


In [101]:
dist_mat = res
num_vertices = mat.shape[0]
available = [True for _ in range(num_vertices)]
expected_components = [np.array([0,1,2]), np.array([3,4,5,6])]

components = []

# finish this loop
while any(available):
    current_node = available.index(True)

    components.append(np.where(dist_mat[current_node] < np.inf)[0].tolist())
    available[current_node] = False

components = np.unique(components)
print(components)
print(expected_components)

[list([0, 1, 2]) list([3, 4, 5, 6])]
[array([0, 1, 2]), array([3, 4, 5, 6])]


In [102]:
short_mat = np.array([[0, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 2],
       [1, 1, 1, 1, 2, 0 ,1],
       [1, 1, 1, 1, 1, 1, 0]])

# count of how many shortest parths use that edge for each starting vertex
# there are 7 vertices in this matrix. so matrix is 7x7, but not all vertex pairs will have an
# edge, so those will be 0
count = np.array([[0, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 1, 1],
       [1, 1, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 0, 1, 2],
       [1, 1, 1, 1, 2, 0 ,1],
       [1, 1, 1, 1, 1, 1, 0]])

In [330]:
# implement breadth-first search and at each vertex, assign the number of shortest paths from the root vertex to that vertex in an array "num_paths"

# at the end of the BFS, the leaf vertices will have 0 unvisited adjacenter vertices. These are the vertices you do the reverse BFS to assign credits to each edge, in a matrix called "vert_credit".

# start a new BFS at each leaf vertex



vertex = 4

# shift list of vertices by starting vertex
num_vertices = mat.shape[0]

res = np.full((num_vertices, num_vertices), np.inf)
edge_count = np.full((num_vertices, num_vertices), 0)



for i in vertices:
    
    Q = deque()
    leaf = []
    prev_adj_vert = [i]
    parent = i

    # initialize visited and layer vectors
    visited = np.full((num_vertices), False)
    layer = np.full((num_vertices), np.inf)

    # set diagonal elements to path of 1
    np.fill_diagonal(res,1)

    # set first vertex in queue and set as visited
    Q.append([i, 1])
    visited[i] = True
    layer[i] = 1

    # run through network while there are still vertices left in the queue
    while Q:
        # current reference number of shortest paths
        path = Q[0][1]

        # determine adjacent vertices
        adj_vert = np.where(mat[Q[0][0]] > 0)[0]

        # filter out parent and sibling vertices
        adj_vert = [vert for vert in adj_vert if layer[vert] > layer[Q[0][0]]]
                
        # run if there exists any viable adjacent vertices
        if adj_vert:
            
            # record the number of shortest paths to the vertices and their layer
            [np.put(res[i], j, path) if res[i][j] == np.inf else np.put(res[i], j, res[i][j] + path) for j in adj_vert]

            for k in adj_vert:
                layer[k] = layer[Q[0][0]]+1 

            # add adjacent vertices to queue and set them as visited
            [Q.append([vert, res[i][vert]]) for vert in adj_vert]
            np.put(visited, adj_vert, True)
            
            # remove current vertex from queue
            Q.popleft()
        
        # if there is no viable adjacent vertex, set this vertex as a leaf
        else:
            leaf.append(Q[0][0])
            Q.popleft()




    ## calculate edge counts
    layer_copy = layer
    layer_copy = sorted(set(layer_copy), reverse=True)
    layer_copy.remove(layer_copy[0])
    
    # at each layer, except bottom-most one
    for j in layer_copy:

        # for every vertex that is in this layer
        for v, l in np.ndenumerate(layer):
            if l == j:
                
                # for every vertex in lower layer that is adjacent to this vertex
                for v_lower, l_lower in np.ndenumerate(layer):                 
                    if (l_lower == l+1 and mat[v][v_lower] == 1):
                        # divide shortest path count of current layer vertex (v) by that of the lower layer vertex (v_lower) and assign that edge_count value to the edge connecting these vertices
                        edge_count[v][v_lower] = edge_count[v][v_lower] + (res[i][v] / res[i][v_lower])
                        #edge_count[v_lower][v] = edge_count[v][v_lower]
                        #print(res[i][v],res[i][v_lower], res[i][v]/res[i][v_lower])


res = np.concatenate((res[vertex:num_vertices + 1], res[0:vertex]))
#print(list(res))
#print(edge_count)


In [304]:
a = [1,2]
b = [2,3]
c = [3,4]
d = [4,5]
e = a,b,c,d
print(e)
e = e[2:4] + e[0:2]
print(e)

([1, 2], [2, 3], [3, 4], [4, 5])
([3, 4], [4, 5], [1, 2], [2, 3])


False